#Import Library and data

In [ ]:
#import Library
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

#Random Forest Model
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#import data from github

#List_Company_train ชุดข้อมูลไว้สำหรับ train
url_train = 'https://raw.githubusercontent.com/NutnichaFah/DSI314-Project-Group8/main/List_Company_train.csv'

#List_company_test ชุดข้อมูลไว้สำหรับ test
url_test = 'https://raw.githubusercontent.com/NutnichaFah/DSI314-Project-Group8/main/List_Company_test.csv'

data = pd.read_csv(url_train)
data_test = pd.read_csv(url_test)

#1. Exploratory Data Analysis (EDA)

In [ ]:
data

In [ ]:
# ดูภาพรวมของข้อมูล
data.info()

In [ ]:
#ตรวจสอบจำนวน missing value
null_cols_values = data.isnull().sum()
null_datapoints = null_cols_values[null_cols_values>0].sum()

print(null_cols_values)
print(f"sum of Missing Value = {null_datapoints}")

In [ ]:
# ตรวจสอบค่า Unique
data.nunique()

In [ ]:
#สร้างฟังก์ชัน เพื่อสร้างกราฟ
def val_count_BarPlot(data=None, count_cols=None):
  data_count = data[count_cols].value_counts()
  fig, ax = plt.subplots(figsize=(10, 8))
  pbar = ax.bar(data_count.index.astype(str),data_count.values)
  ax.set_title(f"{count_cols} values count")

def val_count_BarhPlot(data=None, count_cols=None):
  data_count = data[count_cols].value_counts()
  fig, ax = plt.subplots(figsize=(10, 8))
  pbar = ax.barh(data_count.index.astype(str),data_count.values)
  ax.set_title(f"{count_cols} values count")

  plt.show()

In [ ]:
#กราฟแท่งของคอลัมน์ has_domain
val_count_BarPlot(data=data, count_cols='has_domain')

In [ ]:
data.has_domain.value_counts()

In [ ]:
#กราฟแท่งของคอลัมน์ has_facebook
val_count_BarPlot(data=data, count_cols='has_facebook')

In [ ]:
data.has_facebook.value_counts()

In [ ]:
#กราฟแท่งของคอลัมน์ has_instargram
val_count_BarPlot(data=data, count_cols='has_instargram')

In [ ]:
data.has_instargram.value_counts()

In [ ]:
#กราฟแท่งของคอลัมน์ country_code
val_count_BarhPlot(data=data, count_cols='country_code')

In [ ]:
data.country_code.value_counts()

-------------------------

#2. Feature Engineering

In [ ]:
#เติมค่าว่างในคอลัมน์ has_domain,has_instargram,has_facebook
data['has_domain'] = data['has_domain'].fillna('no')
data['has_instargram'] = data['has_instargram'].fillna('no') 
data['has_facebook'] = data['has_facebook'].fillna('no') 

In [ ]:
#เติมค่าว่างในคอลัมน์ num_employee ด้วยค่า mode
data['num_employee'] = data['num_employee'].fillna(data['num_employee'].mode()[0])

In [ ]:
#เติมค่าว่างในคอลัมน์ web_globalrank ด้วยค่า 0
data['web_globalrank'] = data['web_globalrank'].fillna(0)

In [ ]:
data

In [ ]:
#ในข้อมูลมีประเทศที่ไม่ต้องการ คือ ประเทศที่มีรหัส 364,376

#เรียกดูแถวที่มีประเทศที่มีรหัส 364
data[data['country_code'] == 364]
#ดรอปแถวที่มีประเทศที่มีรหัส 364
data.drop(data.index[[122,123,124,125,126]], inplace=True)
#ดรอปแถวที่มีประเทศที่มีรหัส 376
data = data.drop(labels=[127,128,129,130,131,132,133,134,135,136,137], axis=0)

#ดรอปคอลัมน์ที่ไม่ได้ใช้
data.drop(columns = ['income ($M)','Company_Name','num_employee','incorporation_date','Period'],inplace = True)

data

In [ ]:
#Label encoding คอลัมน์ has_domain, has_instargram, has_facebook ของชุดข้อมูล data
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

data['has_domain']= label_encoder.fit_transform(data['has_domain'])
data['has_instargram']= label_encoder.fit_transform(data['has_instargram'])
data['has_facebook']= label_encoder.fit_transform(data['has_facebook'])
data['Status']= label_encoder.fit_transform(data['Status'])

data


ทำ Feature Engineering กับชุดข้อมูล data_test

In [ ]:
data_test

In [ ]:
data_test['web_globalrank'] = data_test['web_globalrank'].fillna(0)

In [ ]:
data_test.drop(columns = ['Company_Name'],inplace = True)

In [ ]:
data_test

In [ ]:
#Label encoding data_test
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

data_test['has_domain']= label_encoder.fit_transform(data_test['has_domain'])
data_test['has_instargram']= label_encoder.fit_transform(data_test['has_instargram'])
data_test['has_facebook']= label_encoder.fit_transform(data_test['has_facebook'])

In [ ]:
data_test

#3. Data Preparation

In [ ]:
# import library สำหรับการทำ model
from sklearn.model_selection import train_test_split

X = data.drop(columns = ['Status'], axis=1)
y = data['Status']

In [ ]:
# แบ่งชุดข้อมูลเป็น train และ test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

#4. Model

In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_CLF = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=120)

# เทรนโมเดล
rf_CLF.fit(x_train, y_train)

In [ ]:
# ทำนายผล
y_predict = rf_CLF.predict(x_test)
y_predict

In [ ]:
# สร้างฟังก์ชันสำหรับการประเมินโมเดล
from sklearn import metrics

def modelEvaluation(Y_test=None, predictions=None):
  accuracy = metrics.accuracy_score(Y_test, predictions)
  precision = metrics.precision_score(Y_test, predictions)
  recall = metrics.recall_score(Y_test, predictions)
  f1 = metrics.f1_score(Y_test, predictions)

  print(f"Accuracy = {np.round(accuracy, 4)}")
  print(f"Precision = {np.round(precision, 4)}")
  print(f"recall = {np.round(recall, 4)}")
  print(f"F1 = {np.round(f1, 4)}")

In [ ]:
modelEvaluation(Y_test=y_test, predictions=y_predict)

นำโมเดลที่เราได้มาทำนายผล โดยใช้ชุดข้อมูล data_test

In [ ]:
satu_predicts = rf_CLF.predict(data_test)
satu_predicts

#Confusion Matrix

In [ ]:
metrics.confusion_matrix(y_test, y_predict)

In [ ]:
print(metrics.classification_report(y_test, y_predict))